In [33]:
from matplotlib import pyplot as plt
import matplotlib.lines as mlines
import numpy as np
plt.rcParams["figure.figsize"] = (20,10)

In [34]:
class Poly:
    """"
        c_0 + c_1 x + c_2 x^2 + .. + c_deg x^deg
    """
    def __init__(self, *, deg=2, coeffs=None):
        if coeffs is not None:
            self.coeffs = np.array(coeffs)
        else:
            self.coeffs = np.ones(deg+1)

    def __call__(self, x):
        return sum(c_i * x**i for (i, c_i) in enumerate(self.coeffs))

    def grad(self, x):
        return np.array([x**i for (i, c_ip1) in enumerate(self.coeffs[:])]).T[0]

    def hessian(self, x):
        return np.diag(list(x**i for (i, c_ip1) in enumerate(self.coeffs[:])))

In [35]:
def regression(x, y, method, **config):
    if config == {}:
        config = {"lr0": 0.5, "d": 0.005, "epoch": 1000}
    f = lambda x_poly: y - x_poly(x.T[0])
    jacobi = lambda x_poly: np.array([- x_poly.grad(x[i]) for i in range(len(x))])
    bs = method(f, np.zeros(len(x)), **config, jacobi=jacobi)
    # print('hm')
    # print(f'came close by {f(Poly(coeffs=bs[-1]))}')
    return bs[-1]

def gauss_newton(f, x, *, lr, epoch, jacobi):
    points = np.zeros((epoch, len(x)))
    x_poly = Poly(coeffs=x)
    points[0] = x_poly.coeffs

    for i in range(epoch):
        j = jacobi(x_poly)
        g = np.matmul(j.T, f(x_poly).reshape(-1,1))
        h = np.matmul(j.T,j)
        p = np.matmul(np.linalg.inv(h), g).T[0]
        x_poly.coeffs -= lr * p
        points[i] = x_poly.coeffs
    return points

def test_gn(coeffs, points, **config):
    coeffs = np.array(coeffs)
    points = np.array(points)
    test_poly = Poly(coeffs=coeffs)
    res = regression(np.array(points.reshape(-1,1)),test_poly(points),gauss_newton, **config)
    print(f'GN result for {coeffs} is\n{res}')
    return res

regression(np.array([[-2], [3], [5], [4]]), np.array([5, 10, 26, 17]), gauss_newton, lr=1, epoch=1)

array([ 1.00000000e+00, -4.40536496e-13,  1.00000000e+00,  2.84217094e-14])

In [41]:
def dogleg_method_step(grad_k, hessian_k, trust_radius):
    hessian_k_inv = np.linalg.inv(hessian_k)
    dx_newton = -np.matmul(hessian_k_inv, grad_k)
    dx_newton_norm = np.linalg.norm(dx_newton)

    if dx_newton_norm <= trust_radius:
        return dx_newton

    dx_steepest = - np.dot(grad_k, grad_k) / np.dot(grad_k, np.dot(hessian_k,grad_k)) * grad_k
    dx_steepest_norm = np.linalg.norm(dx_steepest)

    if dx_steepest_norm >= trust_radius:
        return trust_radius * dx_steepest / dx_steepest_norm

    diff = dx_newton - dx_steepest
    dx_steepest_x_diff = np.matmul(dx_steepest.T, diff)
    discriminant = dx_steepest_x_diff ** 2 - np.linalg.norm(diff) ** 2 * \
                   (np.linalg.norm(dx_steepest) ** 2 - trust_radius ** 2)
    tau = (-dx_steepest_x_diff + np.sqrt(discriminant)) / np.linalg.norm(diff) ** 2
    return dx_steepest + tau * (dx_newton - dx_steepest)

def trust_region_method(func, grad, hessian, x, tr0=1, tr_limit=2 ** 5, epoch=10, eta=0.1):
    x_poly = Poly(coeffs=x)
    points = np.zeros((epoch, len(x)))
    points[0] = x_poly.coeffs
    trust_radius = tr0
    for i in range(1, epoch):
        grad_k = grad(x_poly)
        hessian_k = hessian(x_poly)
        pk = dogleg_method_step(grad_k, hessian_k, trust_radius)

        moved = Poly(coeffs=x_poly.coeffs + pk)

        # Actual reduction.
        act_red = sum(func(x_poly)**2) - sum(func(moved)**2)

        # Predicted reduction.
        # pred_red = -(np.dot(grad_k, pk) + 0.5 * np.dot(pk, np.dot(hessian_k , pk)))
        pred_red = -(np.matmul(grad_k.T, pk) + 0.5 * np.matmul(pk.T, np.dot(hessian_k, pk)))
        # print(f'{pred_red=}\n{act_red=}')
        # print(f'{trust_radius = }')
        # Rho.
        if pred_red == 0.0:
            rhok = 1e99
        else:
            rhok = act_red / pred_red

        # Calculate the Euclidean norm of pk.
        norm_pk = np.linalg.norm(pk)

        # Rho is close to zero or negative, therefore the trust region is shrunk.
        if rhok < 0.25:
            trust_radius = 0.25 * trust_radius
        else:
            # Rho is close to one and pk has reached the boundary of the trust region, therefore the trust region is expanded.
            if rhok > 0.75 and norm_pk == trust_radius:
                trust_radius = min(2.0 * trust_radius, tr_limit)
            else:
                trust_radius = trust_radius

        # Choose the position for the next iteration.
        if rhok > eta:
            x_poly = moved
        else:
            x_poly = x_poly
        points[i] = x_poly.coeffs
    return points

def regression_pdl(x, y, method, **config):
    if config == {}:
        config = {"lr0": 0.5, "d": 0.005, "epoch": 1000}
    f = lambda x_poly: (y - x_poly(x.T[0]))
    jacobi = lambda x_poly: np.array([- x_poly.grad(x[i]) for i in range(len(x))])
    hessian = lambda x_poly: np.matmul(jacobi(x_poly).T, jacobi(x_poly))
    grad = lambda x_poly: 2*np.matmul(jacobi(x_poly).T, f(x_poly))
    bs = method(f, grad, hessian, np.zeros(len(x)), **config)
    # print('hm')
    # print(f'came close by {f(Poly(coeffs=bs[-1]))}')
    return bs[-1]

def test_pdl(coeffs, points, **config):
    coeffs = np.array(coeffs)
    points = np.array(points)
    test_poly = Poly(coeffs=coeffs)
    res = regression_pdl(np.array(points.reshape(-1,1)),test_poly(points),trust_region_method, **config)
    print(f'PDL result for {coeffs} is\n{res}')
    return res


test_pdl([1, 0, 1], [1, 0, -1], epoch=40, tr0=1)
test_gn([1, 0, 1], [1, 0, -1], lr=1, epoch=1)
test_pdl([1, 0, 1], [1, 0, -1, 2, 3], epoch=40, tr0=1, eta=0.05)
test_gn([1, 0, 1], [1, 0, -1, 2, 3], lr=1, epoch=1)
test_pdl([1, 0, 1, 0, 1, 2], [1, 0, -1, 2, 3, -4], epoch=100, tr0=1, eta=0.1)
test_gn([1, 0, 1, 0, 1, 2], [1, 0, -1, 2, 3, -4], lr=1, epoch=1)

PDL result for [1 0 1] is
[0.99999952 0.         0.9999995 ]
GN result for [1 0 1] is
[1. 0. 1.]
PDL result for [1 0 1] is
[ 1.00005446e+00 -4.82854548e-06  1.00005082e+00 -8.91550443e-06
 -3.41803695e-06]
GN result for [1 0 1] is
[ 1.00000000e+00  3.81028542e-13  1.00000000e+00 -1.84963156e-13
  3.54161145e-14]
PDL result for [1 0 1 0 1 2] is
[9.79021546e-01 6.96281223e-04 9.81089080e-01 5.48161037e-03
 1.00035233e+00 1.99948238e+00]
GN result for [1 0 1 0 1 2] is
[ 9.99999998e-01 -2.61934474e-10  1.00000000e+00  0.00000000e+00
  1.00000000e+00  2.00000000e+00]


array([ 9.99999998e-01, -2.61934474e-10,  1.00000000e+00,  0.00000000e+00,
        1.00000000e+00,  2.00000000e+00])

In [37]:

def draw_2d(x, y, bs):
    x = x.reshape(len(x))
    ax = plt.figure().add_subplot()
    ax.scatter(x, y)
    ax.grid(True)
    tmin = x.min() - 1
    tmax = x.max() + 1
    X = np.array([tmin, tmax])
    Y = (lambda z: bs[0] + bs[1] * z)(X)
    ax.add_line(mlines.Line2D(X, Y, color='green'))

def draw_polynom(x_poly, x_points, y_points=None):
    if y_points is None:
        y_points = x_poly(x_points)
    ax = plt.figure().add_subplot()
    ax.scatter(x_points, y_points)
    ax.grid()
    X = np.linspace(x_points.min(), x_points.max(), 100)
    ax.plot(X, x_poly(X))